# Bitcoin Improvement Proposal 322 - Generic Signed Message Format

## Verification Notebook

This notebook verifies a BIP 0322 simple signature on the message "Hello World" using address `bc1q9vza2e8x573nczrlzms0wvx3gsqjx7vavgkx0l` which is listed as a test vector with a coresponding signature in the BIP.

The notebook attempts to verify both the signature produced in the [BIP0322_signing](http://localhost:8888/notebooks/BIP0322_signing.ipynb) notebook and the signature test vector provided in BIP 0322. It then goes on to contrast this implementation with results from the [P.R.](https://github.com/bitcoin/bitcoin/pull/24058) to add BIP 0322 to Bitcoin core demonstrating that the current test vector in BIP 0322 is incorrect, whilst the signature from the signing notebook is correct (although different from the one initially included in the P.R.'s test suite.

### A Simple Signature


From the BIP-0322:

`A simple signature consists of a witness stack, consensus encoded as a vector of vectors of bytes, and base64-encoded. Validators should construct to_spend and to_sign as defined below, with default values for all fields except that`

## Notebook Author

* Will Abramson
* [Legendary Requirements](http://legreq.com/)
* Contact: will@legreq.com

## Acknowledgements

This work was funded by Ryan Grant and Digital Contract Design. Thanks also go to Joe Andrieu, Kalle Alm, Pieter Wuille and Jimmy Song for engaging with and supporting various aspects of this work.


# Notebook Steps

1. Define the inputs to the BIP 0322 signature verification
2. BIP 0322 Verification Process
    1. Create the to_spend tx
    2. Create the to_sign tx
    3. Verify the signature on the to_sign tx
3. Compare with the P.R. that adds BIP 0322 into Bitcoin Core

## 1. Define Inputs

Inputs include

- The message being signed
- The address that it is being claimed signed the message
- The signature to be verified

### 1.1. Message Signed

The message signed is "Hello World" which is then turned into a tagged hash using the BIP 0322 defined tag. The resulting hash is provided as a test vector in BIP 0322 and verified here.

In [5]:
from buidl.hash import tagged_hash
from buidl.helper import hash256
message = b"Hello World"
tag = b"BIP0322-signed-message"

tagged_hash = tagged_hash(tag,message)

# This test vector is provided in bip0322
test_vector = 'f0eb03b1a75ac6d9847f55c624a99169b5dccba2a31f5b23bea77ba270de0a7a'
assert(test_vector == tagged_hash.hex())

### 1.2. Claimed Address that Signed Message

This address is typically associated with a private key which it is claimed was used to sign the message

In [6]:
# Test vector taken from the bip
address = 'bc1q9vza2e8x573nczrlzms0wvx3gsqjx7vavgkx0l'

### 1.3. Test Vector Sig

This signature is the test vector taken from [bip0322](https://github.com/bitcoin/bips/blob/master/bip-0322.mediawiki#message-signing).

**Currently unable to get this to verify.**

**It appears this is a mistake. It is not the test vector they use in the Bitcoin core P.R. tests**

In [7]:
simple_signature = 'AkcwRAIgG3PASL/vRTgAqogWT6S8rUOQXNnfRzX6JncmbFlHc1ACIGQdsW+rnVmsQzyAYRQisHKFMigDmKiL7LUw4x17Fw5tASECx/EgAxlkQpQ9hYjgGu6EBCPMVPwVIVJqO4XCsMvViHI='

### 1.4. Notebook Produced Sig

This signature was produced in the [BIP0322_signing](http://localhost:8888/notebooks/BIP0322_signing.ipynb) notebook. Currently it is this signature that verifies against the message and signing address.

**Note: It seems likely I  just replicating the same mistake across both notebooks?**

In [8]:
produced_sig = 'AkgwRQIhAOzyynlqt93lOKJr+wmmxIens//zPzl9tqIOua93wO6MAiBi5n5EyAcPScOjf1lAqIUIQtr3zKNeavYabHyR8eGhowEhAsfxIAMZZEKUPYWI4BruhAQjzFT8FSFSajuFwrDL1Yhy'

## 2. BIP 0322 Verification

### A. Compute virtual to_spend transaction

Note: the message_hash and message_challenge are both calculable by the verifier based on the inputs provided. Namely the message being signed and the address which it is claimed was used to sign this message.

```
    nVersion = 0
    nLockTime = 0
    vin[0].prevout.hash = 0000...000
    vin[0].prevout.n = 0xFFFFFFFF
    vin[0].nSequence = 0
    vin[0].scriptSig = OP_0 PUSH32[ message_hash ]
    vin[0].scriptWitness = []
    vout[0].nValue = 0
    vout[0].scriptPubKey = message_challenge
```

In [19]:
from buidl.tx import Tx, TxIn, TxOut
from buidl.helper import little_endian_to_int
from buidl.script import Script,ScriptPubKey

In [20]:
# Not a valid Tx hash. Will never be spendable on any BTC network.
prevout_hash = bytes.fromhex('0000000000000000000000000000000000000000000000000000000000000000')
# prevout.n
prevout_index = little_endian_to_int(bytes.fromhex('FFFFFFFF'))
sequence = 0

# Byte array of message hash
message_hash = tagged_hash

# Spotted the error
# OP_PUSH32 gets automatically added to the stack when add 32 bytes of data
# commands = [0, 32, message_hash]
commands = [0, message_hash]
scriptSig = Script(commands)

# This is equivalent to the one provided
print(scriptSig.raw_serialize().hex())

assert("0020f0eb03b1a75ac6d9847f55c624a99169b5dccba2a31f5b23bea77ba270de0a7a" == scriptSig.raw_serialize().hex())

# Create Tx Input
tx_in = TxIn(prevout_hash,prevout_index,scriptSig,sequence)

0020f0eb03b1a75ac6d9847f55c624a99169b5dccba2a31f5b23bea77ba270de0a7a


In [21]:
from buidl.script import address_to_script_pubkey

# Value of tx output
value = 0

# Convert address to ScriptPubKey
script_pubkey = address_to_script_pubkey(address)

tx_out = TxOut(value,script_pubkey)


In [22]:
tx_out

0:OP_0 2b05d564e6a7a33c087f16e0f730d1440123799d 

In [23]:
# create transaction
version=0
tx_inputs = [tx_in]
tx_outputs = [tx_out]
locktime=0
network="mainnet"

# Could be false, but using a segwit address. I think this is the "Simple Signature" in BIP-0322
segwit=True

virtual_to_spend_tx = Tx(version,tx_inputs,tx_outputs,locktime,network,segwit)

In [24]:
print(virtual_to_spend_tx.is_coinbase())

True


### B. Create the virtual to_sign transaction

**There are two separate to_sign transactions created in this section. One from the BIP 0322 signature test case and one from the signature generated in the signing notebook.**

Note: the signature in simple format is just the witness stack

```python
    nVersion = 0 or (FULL format only) as appropriate (e.g. 2, for time locks)
    nLockTime = 0 or (FULL format only) as appropriate (for time locks)
    vin[0].prevout.hash = to_spend.txid
    vin[0].prevout.n = 0
    vin[0].nSequence = 0 or (FULL format only) as appropriate (for time locks)
    vin[0].scriptWitness = message_signature
    vout[0].nValue = 0
    vout[0].scriptPubKey = OP_RETURN
```

### B.1. Decode the signature into a witness stack

To do this we:

- Decode from base64
- Use this to instantiate a bytes stream
- Instantiate a Witness by parsing this stream

In [29]:
from buidl.helper import base64_decode
from buidl.ecc import Signature
from buidl.witness import Witness
import io

#### B.1.1 Using bip322 test vector

In [30]:

# Note: Using the test vector I am unable to get the code to verify
decoded_witness_test = base64_decode(simple_signature)

stream = io.BytesIO(decoded_witness_test)
witness_test = Witness.parse(stream)
print(witness_test)

304402201b73c048bfef453800aa88164fa4bcad43905cd9df4735fa2677266c594773500220641db16fab9d59ac433c80611422b0728532280398a88becb530e31d7b170e6d01 02c7f12003196442943d8588e01aee840423cc54fc1521526a3b85c2b0cbd58872 


#### B1.1.2 Using the notebook produced signature

In [32]:
# Using the signature produced in the other notebook does work however
decoded_witness_produced = base64_decode(produced_sig)

stream = io.BytesIO(decoded_witness_produced)
witness_produced = Witness.parse(stream)
print(witness_produced)

3045022100ecf2ca796ab7dde538a26bfb09a6c487a7b3fff33f397db6a20eb9af77c0ee8c022062e67e44c8070f49c3a37f5940a8850842daf7cca35e6af61a6c7c91f1e1a1a301 02c7f12003196442943d8588e01aee840423cc54fc1521526a3b85c2b0cbd58872 


### B.2. Create the to_sign transaction input and populate with the witness

#### B.2.1. Create tx_in using bip322 test vector witness

In [33]:
prevout_hash = virtual_to_spend_tx.hash()
prevout_index = 0
sequence = 0


tx_in_test = TxIn(prevout_hash,prevout_index,sequence=sequence)

# set the witness
tx_in_test.witness = witness_test

tx_in_test.witness

304402201b73c048bfef453800aa88164fa4bcad43905cd9df4735fa2677266c594773500220641db16fab9d59ac433c80611422b0728532280398a88becb530e31d7b170e6d01 02c7f12003196442943d8588e01aee840423cc54fc1521526a3b85c2b0cbd58872 

#### B.2.2. Create tx_in using notebook produced witness

In [34]:
prevout_hash = virtual_to_spend_tx.hash()
prevout_index = 0
sequence = 0

tx_in_produced = TxIn(prevout_hash,prevout_index,sequence=sequence)

# set the witness
tx_in_produced.witness = witness_produced

tx_in_produced.witness

3045022100ecf2ca796ab7dde538a26bfb09a6c487a7b3fff33f397db6a20eb9af77c0ee8c022062e67e44c8070f49c3a37f5940a8850842daf7cca35e6af61a6c7c91f1e1a1a301 02c7f12003196442943d8588e01aee840423cc54fc1521526a3b85c2b0cbd58872 

### B.3. Create tx_out (same output for all)

In [36]:
value = 0
# OP Code 106 for OP_RETURN
commands = [106]
scriptPubKey = Script(commands)

tx_output = TxOut(value,scriptPubKey)
tx_output

0:OP_RETURN 

### B.4. Create the virtual to_sign transaction

#### B.4.1. Create virtual_to_sign_tx for test vector

In [37]:
version = 0
locktime = 0
virtual_to_sign_tx_test_vector = Tx(version, [tx_in_test], [tx_output],locktime,segwit=False)

print(virtual_to_sign_tx_test_vector.id())

88737ae86f2077145f93cc4b153ae9a1cb8d56afa511988c149c5c8c9d93bddf


#### B.4.2 Create virtual_to_sign_tx for produced signature

In [38]:
version = 0
locktime = 0
virtual_to_sign_tx_produced = Tx(version, [tx_in_produced], [tx_output],locktime,segwit=False)
print(virtual_to_sign_tx_test_vector.id())

88737ae86f2077145f93cc4b153ae9a1cb8d56afa511988c149c5c8c9d93bddf


## Note: The txid's are the same. The different witness does not change the txid because it is not included in the hash

### A whole bunch of validation that SHOULD happen ... See [verification process](https://github.com/bitcoin/bips/blob/master/bip-0322.mediawiki#verification-process)

### Initially all I am trying to do is validate the signature on the input in the to_sign transaction


### The input for the virtual to_sign transation IS the output at index 0 from the virtual to_spend transaction

Normally the relevant information (The outputs script_pubkey and value) would be populated by querying the network for the transaction and fetching it from there. In this case we set those values on the input manually.

## C. Verify the Signature on the to_sign transcation


### C.1. Verify BIP 0322 Test Vector to_sign transaction

#### C.1.1 Maunally Fetch the to_spend transaction and populate the required values for the to_sign tx_input

In [47]:
virtual_to_spend_tx


tx: b79d196740ad5217771c1098fc4a4b51e0535c32236c71f1ea4d61a2d603352b
version: 0
locktime: 0
tx_ins:
0000000000000000000000000000000000000000000000000000000000000000:4294967295
tx_outs:
0:OP_0 2b05d564e6a7a33c087f16e0f730d1440123799d 

In [46]:
virtual_to_sign_tx_test_vector.tx_ins[0]._script_pubkey = virtual_to_spend_tx.tx_outs[0].script_pubkey
virtual_to_sign_tx_test_vector.tx_ins[0]._value = virtual_to_spend_tx.tx_outs[0].amount

#### C.1.2. Verify the input is a valid unlocking of the to_spend output

**This returns false because the current signature test vector is incorrect!**

In [48]:
virtual_to_sign_tx_test_vector.verify_input(0)

False

### C.2. Verify the Produced Signature from the [Signing Notebook](http://localhost:8888/notebooks/BIP0322_signing.ipynb)

#### C.2.1. Maunally Fetch the to_spend transaction and populate the required values for the to_sign tx_input

In [49]:
virtual_to_sign_tx_produced.tx_ins[0]._script_pubkey = virtual_to_spend_tx.tx_outs[0].script_pubkey
virtual_to_sign_tx_produced.tx_ins[0]._value = virtual_to_spend_tx.tx_outs[0].amount

#### C.2.2. Verify the input is a valid unlocking of the to_spend output


In [50]:
virtual_to_sign_tx_produced.verify_input(0)

True

# This gives us confidence that the signature produced in the signing notebook is a correct BIP 0322 signature

## However, It was noticed that this was not the same signature as the one used in the BIP 0322 P.R. tests. So I did some more digging


# 3. Checking Against the BIP 0322 P.R to Bitcoin Core

## [BIP 0322 basic support #24058](https://github.com/bitcoin/bitcoin/pull/24058)

### The Test Case added to the P.R. for message "Hello World" and addess "bc1q9vza2e8x573nczrlzms0wvx3gsqjx7vavgkx0l"

```c
BOOST_CHECK_EQUAL(
    MessageVerify(
        "bc1q9vza2e8x573nczrlzms0wvx3gsqjx7vavgkx0l",
        "AkcwRAIgZRfIY3p7/DoVTty6YZbWS71bc5Vct9p9Fia83eRmw2QCICK/ENGfwLtptFluMGs2KsqoNSk89pO7F29zJLUx9a/sASECx/EgAxlkQpQ9hYjgGu6EBCPMVPwVIVJqO4XCsMvViHI=",
        "Hello World"),
    MessageVerificationResult::OK);
```

### Adding Print Statements to the Code run when executing the above test case

Output generated by adding the following cout print statements to MessageVerifyBIP322. See https://github.com/bitcoin/bitcoin/blob/843ce398a1d4661b9d93160635b46cc0a70d6153/src/util/message.cpp#L48 for original

```c
MessageVerificationResult MessageVerifyBIP322(
    CTxDestination& destination,
    std::vector<unsigned char>& signature,
    const std::string& message,
    MessageVerificationResult legacyError)
{
    auto txs = BIP322Txs::Create(destination, message, legacyError, &signature);
    if (!txs) return legacyError;

    const CTransaction& to_sign = txs->m_to_sign;
    const CTransaction& to_spend = txs->m_to_spend;

    std::cout << "This is the to_spend tx : \n" << to_spend.ToString();
    std::cout << "\nThis is the to_spend tx hash : \n" << to_spend.GetHash().ToString();

    std::cout << "\n\n This is the to_sign tx : \n" << to_sign.ToString();

    std::cout << "\nThis is the to_sign tx hash : \n" << to_sign.GetHash().ToString();

    const CScript scriptSig = to_sign.vin[0].scriptSig;
    const CScriptWitness& witness = to_sign.vin[0].scriptWitness;

    PrecomputedTransactionData txdata;
    txdata.Init(to_sign, {to_spend.vout[0]});
    TransactionSignatureChecker sigcheck(&to_sign, /* nInIn= */ 0, /* amountIn= */ to_spend.vout[0].nValue, txdata, MissingDataBehavior::ASSERT_FAIL);
    sigcheck.RequireSighashAll();

    if (!VerifyScript(scriptSig, to_spend.vout[0].scriptPubKey, &witness, BIP322_REQUIRED_FLAGS, sigcheck)) {
        return MessageVerificationResult::ERR_INVALID;
    }

    // inconclusive checks

    if (to_sign.nVersion != 0 && to_sign.nVersion != 2) {
        return MessageVerificationResult::INCONCLUSIVE;
    }

    if (!VerifyScript(scriptSig, to_spend.vout[0].scriptPubKey, &witness, BIP322_INCONCLUSIVE_FLAGS, sigcheck)) {
        return MessageVerificationResult::INCONCLUSIVE;
    }

    // timelock check
    if (to_sign.nLockTime > 0 || to_sign.vin[0].nSequence > 0) {
        return MessageVerificationResult::OK_TIMELOCKED;
    }



    return MessageVerificationResult::OK;
}
```

### The Resulting Terminal Output

```bash
This is the to_spend tx : 
CTransaction(hash=b79d196740, ver=0, vin.size=1, vout.size=1, nLockTime=0)
    CTxIn(COutPoint(0000000000, 4294967295), coinbase 0020f0eb03b1a75ac6d9847f55c624a99169b5dccba2a31f5b23bea77ba270de0a7a, nSequence=0)
    CScriptWitness()
    CTxOut(nValue=0.00000000, scriptPubKey=00142b05d564e6a7a33c087f16e0f7)

This is the to_spend tx hash : 
b79d196740ad5217771c1098fc4a4b51e0535c32236c71f1ea4d61a2d603352b

 This is the to_sign tx : 
CTransaction(hash=88737ae86f, ver=0, vin.size=1, vout.size=1, nLockTime=0)
    CTxIn(COutPoint(b79d196740, 0), scriptSig=, nSequence=0)
    CScriptWitness(304402206517c8637a7bfc3a154edcba6196d64bbd5b73955cb7da7d1626bcdde466c364022022bf10d19fc0bb69b4596e306b362acaa835293cf693bb176f7324b531f5afec01, 02c7f12003196442943d8588e01aee840423cc54fc1521526a3b85c2b0cbd58872)
    CTxOut(nValue=0.00000000, scriptPubKey=6a)

This is the to_sign tx hash : 
88737ae86f2077145f93cc4b153ae9a1cb8d56afa511988c149c5c8c9d93bddf


## LOGS from Bitcoin Core Test Case Reporting which test was run
info: check MessageVerify( "bc1q9vza2e8x573nczrlzms0wvx3gsqjx7vavgkx0l", "AkcwRAIgZRfIY3p7/DoVTty6YZbWS71bc5Vct9p9Fia83eRmw2QCICK/ENGfwLtptFluMGs2KsqoNSk89pO7F29zJLUx9a/sASECx/EgAxlkQpQ9hYjgGu6EBCPMVPwVIVJqO4XCsMvViHI=", "Hello World") == MessageVerificationResult::OK has passed
```

### Bitcoin core generated tx hashes extracted from the above output


In [17]:
btc_to_spend_tx_hash = "b79d196740ad5217771c1098fc4a4b51e0535c32236c71f1ea4d61a2d603352b"
btx_to_sign_tx_hash = "88737ae86f2077145f93cc4b153ae9a1cb8d56afa511988c149c5c8c9d93bddf"


## Check to_spend tx hash

In [18]:
print(virtual_to_spend_tx.id())
assert(virtual_to_spend_tx.id() == btc_to_spend_tx_hash)

b79d196740ad5217771c1098fc4a4b51e0535c32236c71f1ea4d61a2d603352b


## Check to_sign tx hash

In [51]:
# Note both have valid tx ids because the witness does not affect the hash of the tx
assert(virtual_to_sign_tx_test_vector.id() == btx_to_sign_tx_hash)
assert(virtual_to_sign_tx_produced.id() == btx_to_sign_tx_hash)

## Identified Differences compared to Below Implementation

### The test vector

Most strikingly the test vector that Bitcoin core uses to get a `MessageVerificationResult::OK` on the message "Hello World" is **NOT** the same as the BIP!

See [line 2596 of util_tests.cpp](https://github.com/bitcoin/bitcoin/blob/843ce398a1d4661b9d93160635b46cc0a70d6153/src/test/util_tests.cpp#L2596):

```c
    BOOST_CHECK_EQUAL(
        MessageVerify(
            "bc1q9vza2e8x573nczrlzms0wvx3gsqjx7vavgkx0l",
            "AkcwRAIgZRfIY3p7/DoVTty6YZbWS71bc5Vct9p9Fia83eRmw2QCICK/ENGfwLtptFluMGs2KsqoNSk89pO7F29zJLUx9a/sASECx/EgAxlkQpQ9hYjgGu6EBCPMVPwVIVJqO4XCsMvViHI=",
            "Hello World"),
        MessageVerificationResult::OK);
```


In [52]:
btc_core_test_vector_sig = "AkcwRAIgZRfIY3p7/DoVTty6YZbWS71bc5Vct9p9Fia83eRmw2QCICK/ENGfwLtptFluMGs2KsqoNSk89pO7F29zJLUx9a/sASECx/EgAxlkQpQ9hYjgGu6EBCPMVPwVIVJqO4XCsMvViHI="

assert(simple_signature == btc_core_test_vector_sig)

AssertionError: 

### Although neither is it the same as the signature I produced in the signing notebook

In [53]:
assert(produced_sig == btc_core_test_vector_sig)

AssertionError: 

## to_spend transaction

Bitcoin Core output when printing the to_spend.ToString() result

################################################################################

```bash
CTransaction(hash=b79d196740, ver=0, vin.size=1, vout.size=1, nLockTime=0)
    CTxIn(COutPoint(0000000000, 4294967295), coinbase 0020f0eb03b1a75ac6d9847f55c624a99169b5dccba2a31f5b23bea77ba270de0a7a, nSequence=0)
    CScriptWitness()
    CTxOut(nValue=0.00000000, scriptPubKey=00142b05d564e6a7a33c087f16e0f7)
```
################################################################################

### Differences

**THERE DON'T APPEAR TO BE ANY**

- CTxIn appears as a coinbase in the print, but so is the tx I produced (See below)
- nValue in CTxOut is in decimal. 
    - Don't think this should matter

### to_sign transaction

Bitcoin Core output

##########################################################################

```bash
CTransaction(hash=88737ae86f, ver=0, vin.size=1, vout.size=1, nLockTime=0)
    CTxIn(COutPoint(b79d196740, 0), scriptSig=, nSequence=0)
    CScriptWitness(304402206517c8637a7bfc3a154edcba6196d64bbd5b73955cb7da7d1626bcdde466c364022022bf10d19fc0bb69b4596e306b362acaa835293cf693bb176f7324b531f5afec01, 02c7f12003196442943d8588e01aee840423cc54fc1521526a3b85c2b0cbd58872)
    CTxOut(nValue=0.00000000, scriptPubKey=6a)
```
##########################################################################

### Differences

- The witness (and hence the actual BIP0322 signature is different)

## Verify the BIP 0322 P.R. Sgnature Value

This replicates the verification code from about but uses the signature value found in the P.R. test case for the same message and address

#### Decode signature into witness stack using the btc core test vector witness

In [55]:
from buidl.helper import base64_decode
from buidl.ecc import Signature
from buidl.witness import Witness

# Using the signature produced in the other notebook does work however
decoded_witness_btc = base64_decode(btc_core_test_vector_sig)

import io
stream = io.BytesIO(decoded_witness_btc)
witness_btc = Witness.parse(stream)
print(witness_btc)

304402206517c8637a7bfc3a154edcba6196d64bbd5b73955cb7da7d1626bcdde466c364022022bf10d19fc0bb69b4596e306b362acaa835293cf693bb176f7324b531f5afec01 02c7f12003196442943d8588e01aee840423cc54fc1521526a3b85c2b0cbd58872 


#### Create tx_in using notebook btc_test_vector witness

In [56]:
prevout_hash = virtual_to_spend_tx.hash()
prevout_index = 0
sequence = 0

tx_in_btc_core = TxIn(prevout_hash,prevout_index,sequence=sequence)

# set the witness
tx_in_btc_core.witness = witness_btc

tx_in_btc_core.witness

304402206517c8637a7bfc3a154edcba6196d64bbd5b73955cb7da7d1626bcdde466c364022022bf10d19fc0bb69b4596e306b362acaa835293cf693bb176f7324b531f5afec01 02c7f12003196442943d8588e01aee840423cc54fc1521526a3b85c2b0cbd58872 

#### Create tx_out

In [57]:
value = 0
# OP Code 106 for OP_RETURN
commands = [106]
scriptPubKey = Script(commands)

tx_output = TxOut(value,scriptPubKey)
tx_output

0:OP_RETURN 

#### Create virtual_to_sign_tx for btc_core_test_vector signature

In [58]:
version = 0
locktime = 0
virtual_to_sign_tx_bitcoin_core = Tx(version, [tx_in_btc_core], [tx_output],locktime,segwit=False)
print(virtual_to_sign_tx_bitcoin_core.id())

88737ae86f2077145f93cc4b153ae9a1cb8d56afa511988c149c5c8c9d93bddf


## Verification of Notebook Bitcoin Core Signature

### Manually populate the value and script_pubkey on the to_sign tx_input from the to_spend tx_output

In [61]:
virtual_to_sign_tx_bitcoin_core.tx_ins[0]._script_pubkey = virtual_to_spend_tx.tx_outs[0].script_pubkey
virtual_to_sign_tx_bitcoin_core.tx_ins[0]._value = virtual_to_spend_tx.tx_outs[0].amount

### Verify the input is a valid unlocking of the to_spend output

Not sure why this isn't working for the test vector of BIP-0322

In [62]:
virtual_to_sign_tx_bitcoin_core.verify_input(0)

True

# This also verifies as true. To double check, I added an additional test case to the BIP 0322 P.R. code in Bitcoin Core for the signature I produced in the signing notebook. This test passed and has since been included as part of the P.R.

## Initially, I thought this must be an error. But because the signature scheme includes randomess, it is actually valid for two signatures to be verifiable for the same message and address pair. See response to my [message](https://github.com/bitcoin/bitcoin/pull/24058/files#r872561862) on the code in the P.R. 